In [20]:
import os
import requests
from langchain_core.tools import tool
@tool
def get_conversion_factor(base_currency:str, target_currency:str): 
    """
    This function fetches the current conversion factor between a given currency and a target currency
    """
    url=f"https://v6.exchangerate-api.com/v6/{os.getenv('EXCHANGE_API_KEY')}/pair/{base_currency}/{target_currency}"
    response=requests.get(url)
    return response.json()

@tool
def convert(base_currency_value:int, conversion_rate:float)->float:
    """
    Given a currency conversion rate this function calculates the target currency value from a given base currency value
    """
    return base_currency_value*conversion_rate

In [25]:
conversion_rate=get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

In [26]:
conversion_rate['conversion_rate']

87.6122

In [31]:
convert.invoke({"base_currency_value":10,"conversion_rate":conversion_rate["conversion_rate"]})

876.1220000000001

In [33]:
from langchain_google_vertexai import ChatVertexAI
# tool binding
llm=ChatVertexAI(model_name="gemini-2.5-pro")

In [34]:
llm_with_tools=llm.bind_tools([get_conversion_factor,convert])

In [35]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
messages=[HumanMessage("What is the conversion factor between USD and INR, based on that can you convert 10 USD to INR")]

In [36]:
llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}, '__gemini_function_call_thought_signatures__': {'a0faee8d-4970-4553-9ad3-a992d73cfb6b': 'CqIRAcu98PDmB54I0dYNO99BzrkT0AlyR0/gJSNblANd2JvvF5/TqdyyjKxVg3WirDckKkjHxLbr+erh2iNPaWe3eCOh5oyzKMoil4Lkvh4JJ3dmhQXo+ApCJpHc9xD2Z7N69iYG0egazsc7PACyymVxIjslHp3mHLhWuOKJvzQpOnCwTeBeMkTgG3KmWbNdA3e5j5YCKry9G/bo9V+tsgxmEjs8Zbq7XOhaPxMbPscHS5XvHeA/Y/1WExK3y65OzONSyVKdbAdT/aXCWbC9wjT0xJ91+RH3EURBv3tb3sGkDWrZWGgfsZGiWhT/mtIfoCCvQgrWwP8wzJ7WdWGa2Qv5IWSL308aYr/XneXMF956Xz8gDleo7h0lfUotdd9gFUxy/4N0mVvfDlLp3atNUXHPWNDkHUbjAN0uOyi0Y/JIOXGytQxl3cZeGnApx4RoDfvhMhCkxFHRVyVvTKeT0WQK9TypRE5yVkdmrJSx6LOGzkkFOCM0dTxi9kr0qhkTXQbgLTB0dh6TVQzyGHW3ozrSnkHycJy1jBqeYCWX/vCIBmOR/oD8fW06Q54AGBm2lr8uG3xmekLx3xWA3RRjWfYGh7vh1V1ofUjpqN0yOFSl+dvPQE8rSc7TX79L6xuxvQy6JkJkhWfPVhuWZ9YTgCBm9wegUSPx1z15REPvYBQiyR3ROBWMsP7L15fdZyVYnMOJI55YGQlOAcIOdA+MPh8vgI5IXuB/Em/NEPFYpUmlBrw